!pip install yt-dlp youtube-search-python

In [1]:
import cv2
import torch
import numpy as np
from torchvision.models.video import r3d_18, R3D_18_Weights
from torchvision import transforms
from yt_dlp import YoutubeDL
from youtubesearchpython import VideosSearch
import os
import subprocess

In [6]:
save_dir = r'data/raw_data'

def download(url, save_directory, num):

    out_temp = os.path.join(save_directory, f"video{num}.%(ext)s")
    ytdl_config = {
        'format': 'bestvideo+bestaudio/best',
        'merge_output_format': 'mp4',
        'noplaylist': True,
        'continuedl': True,
        'no_warnings': True,
        'outtmpl': out_temp,
    }

    with YoutubeDL(ytdl_config) as downloader:
        info = downloader.extract_info(url, download = True)
        video_dest = downloader.prepare_filename(info)

def extract_url(keyword_str_list):
    count = 0
    downloaded = []
    for keyword_str in keyword_str_list:
        query = "Men's long jump olmypics " + keyword_str
        vid_search = VideosSearch(query, limit=2)
        results = vid_search.result()
        for video in results['result']:
            try:
                if video['link'] not in downloaded:
                    download(video['link'], save_dir, count)
                    downloaded.append(video['link'])
            except Exception as e:
                print("[ERROR]: " + str(e))
                continue

In [7]:
extract_url(["beijing", "tokyo", "rio", "paris"])

[youtube] Extracting URL: https://www.youtube.com/watch?v=0WZRrvOjfvA
[youtube] 0WZRrvOjfvA: Downloading webpage
[youtube] 0WZRrvOjfvA: Downloading tv client config
[youtube] 0WZRrvOjfvA: Downloading tv player API JSON
[youtube] 0WZRrvOjfvA: Downloading ios player API JSON
[youtube] 0WZRrvOjfvA: Downloading m3u8 information
[info] 0WZRrvOjfvA: Downloading 1 format(s): 399+251
[download] Destination: data/video0.f399.mp4
[download] 100% of  352.77MiB in 00:00:14 at 23.64MiB/s    
[download] Destination: data/video0.f251.webm
[download] 100% of   17.85MiB in 00:00:00 at 24.29MiB/s    
[Merger] Merging formats into "data/video0.mp4"
Deleting original file data/video0.f399.mp4 (pass -k to keep)
Deleting original file data/video0.f251.webm (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=mGjANZAx45k
[youtube] mGjANZAx45k: Downloading webpage
[youtube] mGjANZAx45k: Downloading tv client config
[youtube] mGjANZAx45k: Downloading tv player API JSON
[youtube] mGjANZAx

In [5]:
import os
import subprocess

def get_bitrate(video_path):
    result = subprocess.run(
        [
            'ffprobe', '-v', 'error',
            '-select_streams', 'v:0',
            '-show_entries', 'stream=bit_rate',
            '-of', 'default=noprint_wrappers=1:nokey=1',
            video_path
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    bitrate_str = result.stdout.decode().strip()
    if bitrate_str == '':
        result = subprocess.run(
            [
                'ffprobe', '-v', 'error',
                '-show_entries', 'format=bit_rate',
                '-of', 'default=noprint_wrappers=1:nokey=1',
                video_path
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        bitrate_str = result.stdout.decode().strip()
    try:
        bitrate = int(bitrate_str)
    except:
        bitrate = None
    return bitrate

def get_duration(video_path):
    result = subprocess.run(
        [
            'ffprobe', '-v', 'error',
            '-show_entries', 'format=duration',
            '-of', 'default=noprint_wrappers=1:nokey=1',
            video_path
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    duration_str = result.stdout.decode().strip()
    try:
        duration = float(duration_str)
    except:
        duration = None
    return duration

def split_video_by_size(video_path, output_dir, target_size_mb=50):

    bitrate = get_bitrate(video_path)  
    duration = get_duration(video_path)  

    if bitrate is None or duration is None:
        print(f"Could not get bitrate or duration for {video_path}. Skipping.")
        return

    target_size_bits = target_size_mb * 8 * 1024 * 1024
    duration_per_chunk = target_size_bits / bitrate
    duration_per_chunk = max(1, duration_per_chunk * 0.95)  

    base_name, ext = os.path.splitext(os.path.basename(video_path))
    start = 0
    index = 1

    while start < duration:
        end = min(start + duration_per_chunk, duration)
        output_file = os.path.join(output_dir, f"{base_name}_part{index}{ext}")
        cmd = f"ffmpeg -y -ss {start} -to {end} -i {video_path} -c copy {output_file}".split(' ')
        print(f"Splitting {video_path} from {start:.2f}s to {end:.2f}s into {output_file}")
        subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        start = end
        index += 1

input_dir = 'data/altered_codec'
output_dir = os.path.join('data', 'split_videos')

for file in os.listdir(input_dir):
    if file.endswith(('.mp4')):
        video_path = os.path.join(input_dir, file)
        split_video_by_size(video_path, output_dir, target_size_mb=50)

print("Splitting complete.")


Splitting data/altered_codec/output1.mp4 from 0.00s to 141.06s into data/split_videos/output1_part1.mp4
Splitting data/altered_codec/output0.mp4 from 0.00s to 136.03s into data/split_videos/output0_part1.mp4
Splitting data/altered_codec/output0.mp4 from 136.03s to 272.05s into data/split_videos/output0_part2.mp4
Splitting data/altered_codec/output0.mp4 from 272.05s to 408.08s into data/split_videos/output0_part3.mp4
Splitting data/altered_codec/output0.mp4 from 408.08s to 544.11s into data/split_videos/output0_part4.mp4
Splitting data/altered_codec/output0.mp4 from 544.11s to 680.13s into data/split_videos/output0_part5.mp4
Splitting data/altered_codec/output0.mp4 from 680.13s to 816.16s into data/split_videos/output0_part6.mp4
Splitting data/altered_codec/output0.mp4 from 816.16s to 952.19s into data/split_videos/output0_part7.mp4
Splitting data/altered_codec/output0.mp4 from 952.19s to 1088.21s into data/split_videos/output0_part8.mp4
Splitting data/altered_codec/output0.mp4 from 108

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = r3d_18(weights=R3D_18_Weights.KINETICS400_V1).eval().to(device)


preprocess = transforms.Compose([
    transforms.Resize((112, 112)),
    
    transforms.ToTensor(),
    transforms.Normalize([0.43216, 0.394666, 0.37645],
                         [0.22803, 0.22145, 0.216989])
])

def extract_segment_features(frames):

    tensor = torch.stack([preprocess(f) for f in frames], dim=1) 
    tensor = tensor.unsqueeze(0).to(device)
    with torch.no_grad():
        features = model.stem(tensor)
        features = model.layer1(features)
        features = model.layer2(features)
        features = model.layer3(features)
        features = model.layer4(features)
        features = features.mean([2, 3, 4])  
    return features.cpu().numpy().flatten()

def get_video_segments(video_path, segment_len, frame_skip):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    segments = []
    frames = []
    timestamps = []
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % frame_skip == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(transforms.ToPILImage()(frame_rgb))
            timestamps.append(frame_idx / fps)
            if len(frames) == segment_len:
                segments.append((frames.copy(), timestamps[0], timestamps[-1]))
                frames.pop(0)  
                timestamps.pop(0)
        frame_idx += 1
    cap.release()
    return segments

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [8]:
ref_segments = get_video_segments('data/ref_vid/ref_vid.mp4', 16, 1)
ref_features = [extract_segment_features(seg[0]) for seg in ref_segments]
ref_feature = np.mean(ref_features, axis=0)  

np.save('models/reference_features', ref_feature)

In [3]:
ref_features = np.load('models/reference_features.npy')

def get_timestamp(vid_path, threshold, ref_feature):
    vid_segments = get_video_segments(vid_path, 16, 2)
    print('vid seg: ', len(vid_segments))
    timestamps = []
    
    for frames, start_time, end_time in vid_segments:
        feature = extract_segment_features(frames)
        sim = cosine_similarity(ref_feature, feature)
        if sim > threshold:
            timestamps.append((start_time, end_time))
    return timestamps


def process_merge_timestamps(vid_dir, ref_features):
    to_return = {}
    print(f"[LOG]: Processing {vid_dir}")
    merged_ts = []
    ts = get_timestamp(vid_dir, 0.795, ref_features)
    print(f"[LOG]: Obtained timestamps\n[LOG]: proceeding to merge")
    for timestamp in ts:
        if not merged_ts or timestamp[0] > merged_ts[-1][1]:
            merged_ts.append([timestamp[0], timestamp[1]])
        else:
            merged_ts[-1][1] = timestamp[1]
    to_return[vid_dir] = merged_ts
    print(f"[LOG]: Processed {vid_dir}")
    print(merged_ts)
    return to_return
    
def clip_video(dir_, output_dir, ref_features):
    for file_path in os.listdir(dir_):
        if not file_path.endswith('.mp4'):
            continue
        print(f"[LOG]: Finding clips in {file_path}")
        timestamps_dir = process_merge_timestamps(os.path.join(dir_, file_path), ref_features)
        count = 0
        for timestamp in timestamps_dir[os.path.join(dir_, file_path)]:
            count += 1
            if (timestamp[1]-timestamp[0])<5:
                continue
            command = f"ffmpeg -ss {timestamp[0]} -to {timestamp[1]} -i {os.path.join(dir_, file_path)} -c:v libx264 -crf 18 -preset veryfast -c:a copy {os.path.join(output_dir, str(file_path[:-4]+str(count)+'.mp4'))}"
            print(command)
            subprocess.run(command.split(' '))
        command_shift = f"mv {os.path.join(dir_, file_path)} {os.path.join("data/processed/", file_path)}"
        subprocess.run(command_shift.split(' '))
        print(f"[LOG]: Snipped and moved {file_path} to processed directory")




In [4]:
clip_video(r'data/split_videos', r'data/final_dataset', ref_features)

[LOG]: Finding clips in output4_part30.mp4
[LOG]: Processing data/split_videos/output4_part30.mp4
vid seg:  2083
[LOG]: Obtained timestamps
[LOG]: proceeding to merge
[LOG]: Processed data/split_videos/output4_part30.mp4
[[8.4, 18.0], [84.96, 94.48], [104.56, 111.28], [111.52, 116.56], [142.8, 153.92], [160.72, 167.76]]
ffmpeg -ss 8.4 -to 18.0 -i data/split_videos/output4_part30.mp4 -c:v libx264 -crf 18 -preset veryfast -c:a copy data/final_dataset/output4_part301.mp4
ffmpeg -ss 84.96 -to 94.48 -i data/split_videos/output4_part30.mp4 -c:v libx264 -crf 18 -preset veryfast -c:a copy data/final_dataset/output4_part302.mp4
ffmpeg -ss 104.56 -to 111.28 -i data/split_videos/output4_part30.mp4 -c:v libx264 -crf 18 -preset veryfast -c:a copy data/final_dataset/output4_part303.mp4
ffmpeg -ss 111.52 -to 116.56 -i data/split_videos/output4_part30.mp4 -c:v libx264 -crf 18 -preset veryfast -c:a copy data/final_dataset/output4_part304.mp4
ffmpeg -ss 142.8 -to 153.92 -i data/split_videos/output4_part

ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-2)
  configuration: --prefix=/home/pradyumnvikram/anaconda3 --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --dis

[LOG]: Snipped and moved output4_part30.mp4 to processed directory
[LOG]: Finding clips in output2_part3.mp4
[LOG]: Processing data/split_videos/output2_part3.mp4
vid seg:  2942
[LOG]: Obtained timestamps
[LOG]: proceeding to merge
[LOG]: Processed data/split_videos/output2_part3.mp4
[[28.8, 32.32], [47.28, 57.44], [64.88, 72.0], [102.48, 108.64], [124.24, 125.6], [133.44, 145.52], [173.76, 176.4], [201.68, 206.96], [226.0, 235.84]]
ffmpeg -ss 47.28 -to 57.44 -i data/split_videos/output2_part3.mp4 -c:v libx264 -crf 18 -preset veryfast -c:a copy data/final_dataset/output2_part32.mp4


ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-2)
  configuration: --prefix=/home/pradyumnvikram/anaconda3 --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --dis

ffmpeg -ss 64.88 -to 72.0 -i data/split_videos/output2_part3.mp4 -c:v libx264 -crf 18 -preset veryfast -c:a copy data/final_dataset/output2_part33.mp4


...

frame=  162 fps=0.0 q=-1.0 Lsize=    3501kB time=00:00:06.36 bitrate=4509.7kbits/s speed=10.1x    
video:3499kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.076063%
[libx264 @ 0x63850b548140] frame I:1     Avg QP:18.48  size: 92197
[libx264 @ 0x63850b548140] frame P:60    Avg QP:18.40  size: 33287
[libx264 @ 0x63850b548140] frame B:101   Avg QP:20.74  size: 14777
[libx264 @ 0x63850b548140] consecutive B-frames:  9.9% 14.8% 18.5% 56.8%
[libx264 @ 0x63850b548140] mb I  I16..4: 11.3% 28.9% 59.8%
[libx264 @ 0x63850b548140] mb P  I16..4: 17.9% 23.2% 12.1%  P16..4: 20.8% 11.0%  6.5%  0.0%  0.0%    skip: 8.5%
[libx264 @ 0x63850b548140] mb B  I16..4:  7.3%  6.6%  2.6%  B16..8: 29.4% 11.5%  2.2%  direct: 9.6%  skip:30.9%  L0:41.5% L1:39.8% BI:18.6%
[libx264 @ 0x63850b548140] 8x8 transform intra:42.1% inter:41.5%
[libx264 @ 0x63850b548140] coded y,uvDC,uvAC intra: 48.8% 43.3% 14.0% inter: 19.5% 18.2% 0.8%
[libx264 @ 0x63850b548140] i16 v,h,dc,p: 33% 54%  9%  4%


ffmpeg -ss 149.12 -to 158.72 -i data/split_videos/output4_part12.mp4 -c:v libx264 -crf 18 -preset veryfast -c:a copy data/final_dataset/output4_part124.mp4


frame=  157 fps=0.0 q=23.0 size=    2304kB time=00:00:04.72 bitrate=3998.8kbits/s speed=8.79x    

[LOG]: Snipped and moved output4_part12.mp4 to processed directory


frame=  240 fps=0.0 q=-1.0 Lsize=    4996kB time=00:00:09.48 bitrate=4317.2kbits/s speed=11.9x    
video:4993kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.068637%
[libx264 @ 0x641c70aa8140] frame I:2     Avg QP:15.25  size: 43878
[libx264 @ 0x641c70aa8140] frame P:109   Avg QP:17.62  size: 30305
[libx264 @ 0x641c70aa8140] frame B:129   Avg QP:20.15  size: 13339
[libx264 @ 0x641c70aa8140] consecutive B-frames: 20.0% 20.8% 12.5% 46.7%
[libx264 @ 0x641c70aa8140] mb I  I16..4: 28.1% 47.4% 24.5%
[libx264 @ 0x641c70aa8140] mb P  I16..4: 17.6% 20.2%  6.5%  P16..4: 24.7% 13.5%  7.2%  0.0%  0.0%    skip:10.3%
[libx264 @ 0x641c70aa8140] mb B  I16..4:  6.5%  4.1%  1.4%  B16..8: 31.2% 12.5%  2.2%  direct: 7.7%  skip:34.6%  L0:41.7% L1:40.9% BI:17.4%
[libx264 @ 0x641c70aa8140] 8x8 transform intra:43.1% inter:33.9%
[libx264 @ 0x641c70aa8140] coded y,uvDC,uvAC intra: 39.6% 43.1% 17.5% inter: 19.7% 17.6% 1.3%
[libx264 @ 0x641c70aa8140] i16 v,h,dc,p: 34% 49% 12%  5%
